In [1]:
"""Cell 1: Setup and Imports"""
import sys
import os
import time
import warnings
import numpy as np
from IPython.display import clear_output

warnings.filterwarnings('ignore')

# Ensure src is in path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Clear all qectostim module cache BEFORE importing (fresh start)
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

# Also clear the bytecode cache
import importlib
importlib.invalidate_caches()

# Import testing utilities from qectostim.testing
from qectostim.testing import (
    clear_qectostim_modules,
    test_composite_construction,
    test_code_circuit,
    test_decoder_on_code,
    load_all_decoders,
    STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL,
)

# Import qectostim
from qectostim.codes import discover_all_codes
from qectostim.codes.composite import (
    ConcatenatedTopologicalCSSCode,
    HomologicalProductCode,
    DualCode,
)
from qectostim.codes.abstract_css import CSSCodeWithComplex
from qectostim.decoders import PyMatchingDecoder

# Load all available decoders
decoder_classes = load_all_decoders()

# Try to import ConcatenatedDecoder
try:
    from qectostim.decoders.concatenated_decoder import ConcatenatedDecoder
    HAS_CONCAT_DECODER = True
except ImportError as e:
    HAS_CONCAT_DECODER = False
    print(f"Note: ConcatenatedDecoder not available: {e}")

print("✓ All imports successful")
print(f"  ConcatenatedTopologicalCSSCode: {ConcatenatedTopologicalCSSCode}")
print(f"  HomologicalProductCode: {HomologicalProductCode}")
print(f"  DualCode: {DualCode}")
print(f"  Decoders loaded: {list(decoder_classes.keys())}")
print(f"  ConcatenatedDecoder available: {HAS_CONCAT_DECODER}")

✓ All imports successful
  ConcatenatedTopologicalCSSCode: <class 'qectostim.codes.composite.concatenated.ConcatenatedTopologicalCSSCode'>
  HomologicalProductCode: <class 'qectostim.codes.composite.homological_product.HomologicalProductCode'>
  DualCode: <class 'qectostim.codes.composite.dual.DualCode'>
  Decoders loaded: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat', 'Hierarchical', 'SingleShot']
  ConcatenatedDecoder available: True


In [2]:
"""Cell 2: Discover Building Block Codes"""
clear_output(wait=True)

print("="*70)
print("DISCOVERING BUILDING BLOCK CODES")
print("="*70)

# Discover codes - use smaller size for composite building
all_codes = discover_all_codes(
    max_qubits=50,  # Smaller for composition
    include_qldpc=True,
    include_subsystem=False,  # Skip subsystem for CSS operations
    include_floquet=False,    # Skip floquet for CSS operations
    include_bosonic=False,
    include_qudit=False,
    include_non_css=True,
    timeout_per_code=3.0,
)

# Filter to CSS codes only (required for concatenation and homological product)
css_codes = {}
for name, code in all_codes.items():
    if hasattr(code, 'hx') and hasattr(code, 'hz'):
        # Additional check: must have k > 0
        if code.k > 0:
            # Verify CSS condition: hx @ hz.T == 0 (mod 2)
            try:
                import numpy as np
                from scipy import sparse
                hx = code.hx
                hz = code.hz
                # Handle sparse matrices
                if sparse.issparse(hx):
                    hx = hx.toarray()
                if sparse.issparse(hz):
                    hz = hz.toarray()
                product = (hx @ hz.T) % 2
                if np.any(product != 0):
                    print(f"  Skipping {name}: hx @ hz.T != 0 (not valid CSS)")
                    continue
            except Exception as e:
                print(f"  Skipping {name}: CSS check failed ({e})")
                continue  # Skip if CSS check fails
            css_codes[name] = code

print(f"\nTotal discovered: {len(all_codes)}")
print(f"CSS codes with k>0: {len(css_codes)}")

# Categorize for smart pairing
small_codes = {}   # n <= 10, good for inner codes
medium_codes = {}  # 10 < n <= 30
surface_codes = {} # Surface-like, good for outer codes
other_codes = {}

for name, code in css_codes.items():
    n = code.n
    if 'Surface' in name or 'Toric' in name:
        surface_codes[name] = code
    elif n <= 10:
        small_codes[name] = code
    elif n <= 30:
        medium_codes[name] = code
    else:
        other_codes[name] = code

print(f"\nBy size:")
print(f"  Small (n≤10): {len(small_codes)}")
print(f"  Medium (10<n≤30): {len(medium_codes)}")
print(f"  Surface-like: {len(surface_codes)}")
print(f"  Other: {len(other_codes)}")


# List codes
print("\n" + "-"*70)
print(f"{'Code Name':<40} {'n':>4} {'k':>3} {'d':>3}")
print("-"*70)
for name, code in sorted(css_codes.items(), key=lambda x: x[1].n)[:20]:
    d = code.metadata.get('distance', '?')
    print(f"{name:<40} {code.n:>4} {code.k:>3} {d:>3}")
if len(css_codes) > 20:
    print(f"... and {len(css_codes) - 20} more")

DISCOVERING BUILDING BLOCK CODES

Total discovered: 58
CSS codes with k>0: 53

By size:
  Small (n≤10): 14
  Medium (10<n≤30): 17
  Surface-like: 11
  Other: 11

----------------------------------------------------------------------
Code Name                                   n   k   d
----------------------------------------------------------------------
Repetition_3                                3   1   3
FourQubit422_[[4,2,2]]                      4   2   2
Repetition_5                                5   1   5
C6                                          6   2   2
Steane_713                                  7   1   3
Hamming_CSS_7                               7   1   3
Repetition_7                                7   1   7
TriangularColour_d3                         7   1   3
TetrahedralColorCode                        7   1   ?
Code_832                                    8   3   2
LoopToricCode4D                             8   2   ?
HexagonalColour_d2                          8   

In [7]:
# Initialize hom_results and dual_results if not defined
if 'hom_results' not in dir():
    hom_results = []
if 'dual_results' not in dir():
    dual_results = []

# Also initialize codes_with_complex if needed
if 'codes_with_complex' not in dir():
    codes_with_complex = {}

In [ ]:
"""Cell 6a: Collect Decoder Smoke Test Results (no output)

This cell collects all results for composite code decoder tests.
Results are stored in `smoke_test_results` for display in the next cell.

Structure:
1. [NO CONCATENATION] - Baseline LER for outer codes only
2. [CONCATENATED CODES] - Should have LOWER LER than baseline
3. [MULTI-LEVEL CONCATENATED] - Additional concatenation layer  
4. [BASE-BEFORE-HOMOLOGICAL-PRODUCT] - Baseline for codes before tensor product
5. [HOMOLOGICAL PRODUCT - NO METACHECKS] - Product codes without metachecks
6. [HOMOLOGICAL PRODUCT - WITH METACHECKS] - Product codes with metachecks
7. [DUAL CODES] - Dual code tests
"""
import sys
from IPython.display import clear_output

# Clear qectostim modules to pick up any code fixes
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

# Re-import essential modules
import numpy as np
from qectostim.codes.discovery import discover_all_codes
from qectostim.codes.composite.concatenated import ConcatenatedTopologicalCSSCode
from qectostim.codes.composite import HomologicalProductCode, DualCode
from qectostim.testing import test_decoder_on_code, load_all_decoders, STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL

# Reload decoder classes and code lists
decoder_classes = load_all_decoders()
all_codes = discover_all_codes()
css_codes = {n: c for n, c in all_codes.items() 
             if hasattr(c, 'hx') and hasattr(c, 'hz') and c.k > 0}

# Parameters
P_ERR = 0.00075  # Error rate needs to be subthreshold
SHOTS = 500000  # Increased shots for reliable LER estimates
ROUNDS = 1

# Storage for all results
smoke_test_results = {
    'params': {'p': P_ERR, 'shots': SHOTS, 'rounds': ROUNDS},
    'decoder_names': [],
    'sections': {},  # section_name -> list of row results
    'baselines': {},  # code_name -> best_ler (for comparison)
    'hom_base_lers': {},
    'hom_no_meta_lers': {},
}

print("Collecting decoder smoke test results...")

if len(decoder_classes) == 0:
    print("⚠️ No decoders available")
    smoke_test_results['decoder_names'] = []
else:
    # Use all available decoders (prioritize BPOSD, PyMatching)
    primary_decoders = {}
    for name in ['BPOSD', 'PyMatching']:
        if name in decoder_classes:
            primary_decoders[name] = decoder_classes[name]
    for name, cls in decoder_classes.items():
        if name not in primary_decoders:
            primary_decoders[name] = cls
    
    decoder_names = list(primary_decoders.keys())
    smoke_test_results['decoder_names'] = decoder_names
    
    def test_code_all_decoders(code, code_label, enable_metachecks=False):
        """Test a code with ALL decoders, return dict of results."""
        results = {'label': code_label, 'no_decode': None, 'decoders': {}}
        for dec_name, dec_class in primary_decoders.items():
            try:
                res = test_decoder_on_code(
                    code, dec_class, decoder_name=dec_name,
                    code_type='CSS', p=P_ERR, shots=SHOTS, rounds=ROUNDS,
                    enable_metachecks=enable_metachecks
                )
                if results['no_decode'] is None and res.ler_no_decode is not None:
                    results['no_decode'] = res.ler_no_decode
                results['decoders'][dec_name] = {'ler': res.ler, 'status': res.status}
            except Exception as e:
                results['decoders'][dec_name] = {'ler': None, 'status': 'FAIL', 'error': str(e)[:30]}
        return results
    
    def get_best_ler(results):
        """Get best (minimum) LER from results dict."""
        lers = [r['ler'] for r in results['decoders'].values() if r.get('ler') is not None]
        return min(lers) if lers else None
    
    # =========================================================================
    # 1. [NO CONCATENATION]
    # =========================================================================
    print("  [1/7] NO CONCATENATION...")
    smoke_test_results['sections']['NO_CONCAT'] = []
    
    outer_codes_tested = set()
    if 'concat_results' in dir() or 'concat_results' in globals():
        successful_concats = [r for r in concat_results if r['status'] == 'OK'][:5]
        
        for concat_res in successful_concats:
            outer_name = concat_res['outer']
            if outer_name in outer_codes_tested:
                continue
            outer_codes_tested.add(outer_name)
            
            outer_code = css_codes.get(outer_name)
            if outer_code is None:
                continue
            
            results = test_code_all_decoders(outer_code, outer_name, enable_metachecks=False)
            best_ler = get_best_ler(results)
            if best_ler is not None:
                smoke_test_results['baselines'][outer_name] = best_ler
            results['is_baseline'] = True
            smoke_test_results['sections']['NO_CONCAT'].append(results)
    
    # =========================================================================
    # 2. [CONCATENATED CODES]
    # =========================================================================
    print("  [2/7] CONCATENATED CODES...")
    smoke_test_results['sections']['CONCAT'] = []
    
    if 'concat_results' in dir() or 'concat_results' in globals():
        successful_concats = [r for r in concat_results if r['status'] == 'OK'][:5]
        
        for concat_res in successful_concats:
            outer_name = concat_res['outer']
            inner_name = concat_res['inner']
            
            outer_code = css_codes.get(outer_name)
            inner_code = css_codes.get(inner_name)
            
            if outer_code is None or inner_code is None:
                continue
            
            try:
                concat = ConcatenatedTopologicalCSSCode(outer_code, inner_code)
                code_label = f"{outer_name}∘{inner_name}"
                
                results = test_code_all_decoders(concat, code_label, enable_metachecks=False)
                results['baseline_key'] = outer_name
                smoke_test_results['sections']['CONCAT'].append(results)
                
            except Exception as e:
                smoke_test_results['sections']['CONCAT'].append({
                    'label': f"{outer_name}∘{inner_name}",
                    'error': str(e)[:50]
                })
    
    # =========================================================================
    # 3. [MULTI-LEVEL CONCATENATED]
    # =========================================================================
    print("  [3/7] MULTI-LEVEL CONCATENATED...")
    smoke_test_results['sections']['MULTI_CONCAT'] = []
    
    inner_422 = None
    for name, code in css_codes.items():
        if code.n == 4 and code.k == 2:
            inner_422 = (name, code)
            break
    if inner_422 is None:
        for name, code in sorted(css_codes.items(), key=lambda x: x[1].n):
            if code.k >= 1 and code.n <= 7:
                inner_422 = (name, code)
                break
    
    outer_surface = None
    for name, code in css_codes.items():
        if 'Surface' in name or 'Toric' in name:
            if code.n <= 25:
                outer_surface = (name, code)
                break
    if outer_surface is None:
        for name, code in sorted(css_codes.items(), key=lambda x: x[1].n):
            if code.n >= 9 and code.n <= 30:
                outer_surface = (name, code)
                break
    
    if inner_422 and outer_surface:
        inner_name, inner_code = inner_422
        outer_name, outer_code = outer_surface
        
        try:
            level1 = ConcatenatedTopologicalCSSCode(inner_code, inner_code)
            level1_label = f"({inner_name}∘{inner_name})"
            level2 = ConcatenatedTopologicalCSSCode(outer_code, level1)
            level2_label = f"{outer_name}∘{level1_label}"
            
            results = test_code_all_decoders(level2, level2_label, enable_metachecks=False)
            results['baseline_key'] = outer_name
            smoke_test_results['sections']['MULTI_CONCAT'].append(results)
            
        except Exception as e:
            smoke_test_results['sections']['MULTI_CONCAT'].append({
                'label': 'Multi-level concat',
                'error': str(e)[:50]
            })
    
    # =========================================================================
    # 4. [BASE-BEFORE-HOMOLOGICAL-PRODUCT]
    # =========================================================================
    print("  [4/7] BASE-BEFORE-HOMOLOGICAL-PRODUCT...")
    smoke_test_results['sections']['HOM_BASE'] = []
    
    codes_with_complex_local = {}
    for name, code in css_codes.items():
        if hasattr(code, 'chain_complex') and code.chain_complex is not None:
            codes_with_complex_local[name] = code
    
    try:
        from qectostim.codes.surface.toric_code_general import ToricCode
        tc = ToricCode(Lx=2, Ly=2)
        if hasattr(tc, 'chain_complex') and tc.chain_complex is not None:
            codes_with_complex_local['ToricCode_2x2'] = tc
    except:
        pass
    
    for name, code in list(codes_with_complex_local.items())[:3]:
        results = test_code_all_decoders(code, name, enable_metachecks=False)
        best_ler = get_best_ler(results)
        if best_ler is not None:
            smoke_test_results['hom_base_lers'][name] = best_ler
        results['is_baseline'] = True
        smoke_test_results['sections']['HOM_BASE'].append(results)
    
    # =========================================================================
    # 5. [HOMOLOGICAL PRODUCT - NO METACHECKS]
    # =========================================================================
    print("  [5/7] HOMOLOGICAL PRODUCT - NO METACHECKS...")
    smoke_test_results['sections']['HOM_NO_META'] = []
    
    hom_products = []
    complex_list = list(codes_with_complex_local.items())[:2]
    if len(complex_list) >= 1:
        for name_a, code_a in complex_list:
            try:
                product = HomologicalProductCode(code_a, code_a)
                code_label = f"{name_a}⊗{name_a}"
                hom_products.append((code_label, product, name_a))
                
                results = test_code_all_decoders(product, code_label, enable_metachecks=False)
                results['baseline_key'] = name_a
                best_ler = get_best_ler(results)
                if best_ler is not None:
                    smoke_test_results['hom_no_meta_lers'][code_label] = best_ler
                smoke_test_results['sections']['HOM_NO_META'].append(results)
                
            except Exception as e:
                smoke_test_results['sections']['HOM_NO_META'].append({
                    'label': f"{name_a}⊗{name_a}",
                    'error': str(e)[:50]
                })
    
    # =========================================================================
    # 6. [HOMOLOGICAL PRODUCT - WITH METACHECKS]
    # =========================================================================
    print("  [6/7] HOMOLOGICAL PRODUCT - WITH METACHECKS...")
    smoke_test_results['sections']['HOM_META'] = []
    
    for code_label, product, name_a in hom_products:
        try:
            results = test_code_all_decoders(product, code_label, enable_metachecks=True)
            results['baseline_key'] = code_label  # Compare to NO_META version
            smoke_test_results['sections']['HOM_META'].append(results)
            
        except Exception as e:
            smoke_test_results['sections']['HOM_META'].append({
                'label': code_label,
                'error': str(e)[:50]
            })
    
    # =========================================================================
    # 7. [5-CHAIN HOMOLOGICAL PRODUCTS - SINGLESHOT DECODER TEST]
    # =========================================================================
    print("  [7/9] 5-CHAIN HOMOLOGICAL PRODUCTS (SingleShot test)...")
    smoke_test_results['sections']['HOM_5CHAIN'] = []
    
    # Build 5-chain codes: 3-chain ⊗ 3-chain = 5-chain
    # ToricCode, Steane, Shor all have 3-chain complexes
    five_chain_products = []
    
    # Try ToricCode ⊗ ToricCode
    try:
        from qectostim.codes.surface.toric_code_general import ToricCode
        tc_small = ToricCode(Lx=2, Ly=2)
        if hasattr(tc_small, 'chain_complex') and tc_small.chain_complex is not None:
            product_toric = HomologicalProductCode(tc_small, tc_small)
            five_chain_products.append(('ToricCode_2x2⊗ToricCode_2x2', product_toric, tc_small))
            print(f"    Created ToricCode⊗ToricCode: n={product_toric.n}, chain_length={product_toric.chain_length}")
    except Exception as e:
        print(f"    ToricCode⊗ToricCode failed: {e}")
    
    # Try Steane ⊗ Steane (both have 3-chain from CSSChainComplex3)
    try:
        from qectostim.codes.small.steane_713 import SteaneCode713
        steane = SteaneCode713()
        if hasattr(steane, 'chain_complex') and steane.chain_complex is not None:
            product_steane = HomologicalProductCode(steane, steane)
            five_chain_products.append(('Steane⊗Steane', product_steane, steane))
            print(f"    Created Steane⊗Steane: n={product_steane.n}, chain_length={product_steane.chain_length}")
    except Exception as e:
        print(f"    Steane⊗Steane failed: {e}")
    
    # Test each 5-chain product with metachecks enabled (for SingleShot)
    for code_label, product, base_code in five_chain_products:
        # Check metachecks
        has_meta_x = hasattr(product, 'meta_x') and product.meta_x is not None and product.meta_x.size > 0
        has_meta_z = hasattr(product, 'meta_z') and product.meta_z is not None and product.meta_z.size > 0
        print(f"    {code_label}: meta_x={has_meta_x}, meta_z={has_meta_z}")
        
        try:
            results = test_code_all_decoders(product, code_label, enable_metachecks=True)
            results['has_metachecks'] = has_meta_x or has_meta_z
            results['chain_length'] = product.chain_length if hasattr(product, 'chain_length') else None
            smoke_test_results['sections']['HOM_5CHAIN'].append(results)
        except Exception as e:
            smoke_test_results['sections']['HOM_5CHAIN'].append({
                'label': code_label,
                'error': str(e)[:50]
            })
    
    # =========================================================================
    # 8. [DUAL CODES]
    # =========================================================================
    print("  [8/9] DUAL CODES...")
    smoke_test_results['sections']['DUAL'] = []
    
    if 'dual_results' in dir() or 'dual_results' in globals():
        successful_dual = [r for r in dual_results if r['status'] == 'OK'][:3]
        
        for dual_res in successful_dual:
            orig_name = dual_res['original']
            orig_code = css_codes.get(orig_name)
            
            if orig_code is None:
                continue
            
            try:
                dual = DualCode(orig_code)
                code_label = f"Dual({orig_name})"
                
                results = test_code_all_decoders(dual, code_label, enable_metachecks=False)
                smoke_test_results['sections']['DUAL'].append(results)
                
            except Exception as e:
                smoke_test_results['sections']['DUAL'].append({
                    'label': f"Dual({orig_name})",
                    'error': str(e)[:50]
                })

print("\n✓ Results collected. Run the next cell to display formatted output.")

  [1/7] NO CONCATENATION...
  [2/7] CONCATENATED CODES...
  [3/7] MULTI-LEVEL CONCATENATED...


In [14]:
"""Cell 6b: Display Decoder Smoke Test Results

Formats and displays results collected in the previous cell.
"""
from IPython.display import clear_output
clear_output(wait=True)

import numpy as np

# Check if results exist
if 'smoke_test_results' not in dir() and 'smoke_test_results' not in globals():
    print("⚠️ Run Cell 6a first to collect results")
else:
    params = smoke_test_results['params']
    decoder_names = smoke_test_results['decoder_names']
    sections = smoke_test_results['sections']
    baselines = smoke_test_results['baselines']
    hom_base_lers = smoke_test_results['hom_base_lers']
    hom_no_meta_lers = smoke_test_results['hom_no_meta_lers']
    
    def format_ler_with_arrow(ler, baseline=None):
        """Format LER with trend arrow if baseline provided."""
        if ler is None:
            return "✗fail"
        ler_str = f"{ler:.4f}"
        if baseline is not None and baseline > 0:
            if ler < baseline * 0.5:
                return f"{ler_str} ▼▼"
            elif ler < baseline * 0.9:
                return f"{ler_str} ▼"
            elif ler > baseline * 1.5:
                return f"{ler_str} ▲▲!"
            elif ler > baseline * 1.1:
                return f"{ler_str} ▲"
        return ler_str
    
    def get_best_ler(results):
        """Get best LER from results."""
        if 'decoders' not in results:
            return None
        lers = [r['ler'] for r in results['decoders'].values() if r.get('ler') is not None]
        return min(lers) if lers else None
    
    # Print header
    print("=" * 120)
    print("DECODER SMOKE TEST ON COMPOSITE CODES - COMPREHENSIVE COMPARISON")
    print("=" * 120)
    print(f"\nParameters: p={params['p']}, shots={params['shots']}, rounds={params['rounds']}")
    print("Expectation: More concatenation → LOWER LER; Metachecks → LOWER LER for homological products\n")
    print(f"Testing with decoders: {decoder_names}\n")
    
    # Build header row
    col_width = 12
    code_col_width = 50
    header = f"  {'Code':<{code_col_width}} {'no-dec':>{col_width}}"
    for dec_name in decoder_names:
        header += f" {dec_name[:col_width]:<{col_width}}"
    header += "  Status"
    sep_line = "-" * (code_col_width + 4 + col_width + (col_width + 1) * len(decoder_names) + 15)
    
    print(header)
    print(sep_line)
    
    # Section display info
    section_info = [
        ('NO_CONCAT', '[NO CONCATENATION] - Baseline LER for outer codes', None),
        ('CONCAT', '[CONCATENATED CODES] - Should have LOWER LER than outer code baseline', 'baselines'),
        ('MULTI_CONCAT', '[MULTI-LEVEL CONCATENATED] - Double concatenation (even lower LER expected)', 'baselines'),
        ('HOM_BASE', '[BASE-BEFORE-HOMOLOGICAL-PRODUCT] - Baseline codes for tensor product', None),
        ('HOM_NO_META', '[HOMOLOGICAL PRODUCT - NO METACHECKS] - 4D codes without metachecks', 'hom_base_lers'),
        ('HOM_META', '[HOMOLOGICAL PRODUCT - WITH METACHECKS] - Should be BETTER than no metachecks', 'hom_no_meta_lers'),
        ('HOM_5CHAIN', '[5-CHAIN HOMOLOGICAL PRODUCTS] - ToricCode⊗ToricCode etc. for SingleShot decoder', None),
        ('DUAL', '[DUAL CODES]', None),
    ]
    
    all_section_lers = {}
    
    for sec_key, sec_title, baseline_source in section_info:
        if sec_key not in sections or not sections[sec_key]:
            continue
        
        print(f"\n{sec_title}")
        all_section_lers[sec_key] = []
        
        for results in sections[sec_key]:
            label = results.get('label', '???')
            if len(label) > code_col_width - 2:
                label = label[:code_col_width - 5] + "..."
            
            # Handle error case
            if 'error' in results:
                print(f"  {label:<{code_col_width}} ERROR: {results['error']}")
                continue
            
            # Get baseline for comparison
            baseline = None
            if baseline_source == 'baselines':
                baseline = baselines.get(results.get('baseline_key'))
            elif baseline_source == 'hom_base_lers':
                baseline = hom_base_lers.get(results.get('baseline_key'))
            elif baseline_source == 'hom_no_meta_lers':
                baseline = hom_no_meta_lers.get(results.get('baseline_key'))
            
            # No-decode column
            no_dec = results.get('no_decode')
            no_dec_str = f"{no_dec:.4f}" if no_dec is not None else "---"
            
            row = f"  {label:<{code_col_width}} {no_dec_str:>{col_width}}"
            
            # Each decoder column
            for dec_name in decoder_names:
                dec_res = results.get('decoders', {}).get(dec_name, {})
                ler = dec_res.get('ler')
                if dec_res.get('status') == 'SKIP':
                    row += f" {'-skip':<{col_width}}"
                else:
                    ler_str = format_ler_with_arrow(ler, baseline)
                    row += f" {ler_str:<{col_width}}"
            
            # Status column
            best_ler = get_best_ler(results)
            if best_ler is not None:
                all_section_lers[sec_key].append(best_ler)
            
            is_baseline = results.get('is_baseline', False)
            if is_baseline:
                row += "  baseline"
            elif baseline is not None and best_ler is not None:
                if best_ler < baseline * 0.3:
                    row += f"  {STATUS_OK} MUCH better"
                elif best_ler < baseline * 0.7:
                    row += f"  {STATUS_OK} improved"
                elif best_ler < baseline * 0.95:
                    row += f"  {STATUS_OK} slightly better"
                elif best_ler > baseline * 1.5:
                    row += f"  {STATUS_FAIL} WORSE!"
                elif best_ler > baseline * 1.1:
                    row += f"  {STATUS_WARN} worse"
                else:
                    row += f"  ~ similar"
            else:
                row += "  ---"
            
            print(row)
    
    print(sep_line)
    
    # SUMMARY
    print("\n" + "=" * 80)
    print("SUMMARY")
    print("=" * 80)
    
    print(f"\n{'Section':<40} {'Avg LER':>12} {'Min LER':>12} {'Max LER':>12} {'Count':>8}")
    for sec_key in ['NO_CONCAT', 'CONCAT', 'MULTI_CONCAT', 'HOM_BASE', 'HOM_NO_META', 'HOM_META', 'HOM_5CHAIN', 'DUAL']:
        if sec_key in all_section_lers and all_section_lers[sec_key]:
            lers = all_section_lers[sec_key]
            avg_ler = np.mean(lers)
            min_ler = np.min(lers)
            max_ler = np.max(lers)
            print(f"{sec_key:<40} {avg_ler:>12.4f} {min_ler:>12.4f} {max_ler:>12.4f} {len(lers):>8}")
    
    # Key comparisons
    print("\nKey Comparisons:")
    
    if 'NO_CONCAT' in all_section_lers and 'CONCAT' in all_section_lers:
        base_avg = np.mean(all_section_lers['NO_CONCAT'])
        concat_avg = np.mean(all_section_lers['CONCAT'])
        if base_avg > 0:
            improvement = (base_avg - concat_avg) / base_avg * 100
            status = STATUS_OK if improvement > 20 else (STATUS_WARN if improvement > 0 else STATUS_FAIL)
            print(f"  {status} Concatenation: {base_avg:.4f} → {concat_avg:.4f} ({improvement:+.1f}%)")
    
    if 'NO_CONCAT' in all_section_lers and 'MULTI_CONCAT' in all_section_lers:
        base_avg = np.mean(all_section_lers['NO_CONCAT'])
        multi_avg = np.mean(all_section_lers['MULTI_CONCAT'])
        if base_avg > 0:
            improvement = (base_avg - multi_avg) / base_avg * 100
            status = STATUS_OK if improvement > 40 else (STATUS_WARN if improvement > 0 else STATUS_FAIL)
            print(f"  {status} Multi-concat: {base_avg:.4f} → {multi_avg:.4f} ({improvement:+.1f}%)")
    
    if 'HOM_NO_META' in all_section_lers and 'HOM_META' in all_section_lers:
        no_meta_avg = np.mean(all_section_lers['HOM_NO_META'])
        meta_avg = np.mean(all_section_lers['HOM_META'])
        if no_meta_avg > 0:
            improvement = (no_meta_avg - meta_avg) / no_meta_avg * 100
            status = STATUS_OK if improvement > 10 else (STATUS_WARN if improvement > 0 else STATUS_FAIL)
            print(f"  {status} Metachecks (hom. product): {no_meta_avg:.4f} → {meta_avg:.4f} ({improvement:+.1f}%)")
        elif meta_avg == 0 and no_meta_avg == 0:
            print(f"  {STATUS_OK} Metachecks (hom. product): both 0.0000 (perfect)")
    
    print("\n" + "=" * 80)

DECODER SMOKE TEST ON COMPOSITE CODES - COMPREHENSIVE COMPARISON

Parameters: p=0.00075, shots=500000, rounds=1
Expectation: More concatenation → LOWER LER; Metachecks → LOWER LER for homological products

Testing with decoders: ['BPOSD', 'PyMatching', 'FusionBlossom', 'BeliefMatching', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat', 'Hierarchical', 'SingleShot']

  Code                                                     no-dec BPOSD        PyMatching   FusionBlosso BeliefMatchi Tesseract    UnionFind    MLE          Hypergraph   Chromobius   Concatenated FlatConcat   Hierarchical SingleShot    Status
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

[NO CONCATENATION] - Baseline LER for outer codes
  RotatedSurface_[[9,1,3]]                   

In [10]:
"""Cell 7: Dual Code Tests"""
clear_output(wait=True)

print("="*70)
print("DUAL CODE TESTS")
print("="*70)
print("\nTesting DualCode(code) - swaps X and Z sectors\n")

dual_results = []

# Test on a selection of codes
dual_test_codes = list(css_codes.items())[:10]

for i, (name, code) in enumerate(dual_test_codes):
    print(f"[{i+1}/{len(dual_test_codes)}] Dual({name})...", end=' ', flush=True)
    
    result = {
        'original': name,
        'status': 'UNKNOWN',
        'n': None,
        'k': None,
        'error': None,
    }
    
    try:
        dual = DualCode(code)
        result['n'] = dual.n
        result['k'] = dual.k
        
        # Verify n and k preserved
        if dual.n == code.n and dual.k == code.k:
            result['status'] = 'OK'
        else:
            result['status'] = 'WARN'
            result['error'] = f"n,k changed: {code.n},{code.k} -> {dual.n},{dual.k}"
            
    except Exception as e:
        result['status'] = 'FAIL'
        result['error'] = f"{type(e).__name__}: {str(e)[:30]}"
    
    dual_results.append(result)
    
    if result['status'] == 'OK':
        print(f"{STATUS_OK} n={result['n']}, k={result['k']}")
    elif result['status'] == 'WARN':
        print(f"{STATUS_WARN} {result['error']}")
    else:
        print(f"{STATUS_FAIL} {result['error']}")

# Summary
ok = sum(1 for r in dual_results if r['status'] == 'OK')
warn = sum(1 for r in dual_results if r['status'] == 'WARN')
fail = sum(1 for r in dual_results if r['status'] == 'FAIL')

print("\n" + "-"*70)
print(f"DUAL CODE SUMMARY: {ok} {STATUS_OK} | {warn} {STATUS_WARN} | {fail} {STATUS_FAIL}")

DUAL CODE TESTS

Testing DualCode(code) - swaps X and Z sectors

[1/10] Dual(FourQubit422_[[4,2,2]])... ✓ n=4, k=2
[2/10] Dual(C6)... ✓ n=6, k=2
[3/10] Dual(Steane_713)... ✓ n=7, k=1
[4/10] Dual(Shor_91)... ✓ n=9, k=1
[5/10] Dual(ReedMuller_15_1_3)... ✓ n=15, k=1
[6/10] Dual(Hamming_CSS_7)... ✓ n=7, k=1
[7/10] Dual(Hamming_CSS_15)... ✓ n=15, k=7
[8/10] Dual(Hamming_CSS_31)... ✓ n=31, k=21
[9/10] Dual(Code_832)... ✓ n=8, k=3
[10/10] Dual(Repetition_3)... ✓ n=3, k=1

----------------------------------------------------------------------
DUAL CODE SUMMARY: 10 ✓ | 0 ⚠️ | 0 ✗
✓ n=4, k=2
[2/10] Dual(C6)... ✓ n=6, k=2
[3/10] Dual(Steane_713)... ✓ n=7, k=1
[4/10] Dual(Shor_91)... ✓ n=9, k=1
[5/10] Dual(ReedMuller_15_1_3)... ✓ n=15, k=1
[6/10] Dual(Hamming_CSS_7)... ✓ n=7, k=1
[7/10] Dual(Hamming_CSS_15)... ✓ n=15, k=7
[8/10] Dual(Hamming_CSS_31)... ✓ n=31, k=21
[9/10] Dual(Code_832)... ✓ n=8, k=3
[10/10] Dual(Repetition_3)... ✓ n=3, k=1

--------------------------------------------------------

In [11]:
"""Cell 8: Comprehensive Results Table"""
clear_output(wait=True)

print("="*100)
print("COMPOSITE CODE SMOKE TEST - COMPREHENSIVE RESULTS")
print("="*100)

# Concatenation results
print("\nCONCATENATION (ConcatenatedTopologicalCSSCode):")
print("-"*100)
print(f"{'Outer':<20} | {'Inner':<20} | {'n':>5} | {'k':>3} | {'Circuit':>8} | Status")
print("-"*100)

for r in concat_results:
    outer = r['outer'][:20] if r['outer'] else 'N/A'
    inner = r['inner'][:20] if r['inner'] else 'N/A'
    n = str(r['n']) if r['n'] else '-'
    k = str(r['k']) if r['k'] else '-'
    circ = STATUS_OK if r.get('circuit_ok') else STATUS_FAIL
    
    if r['status'] == 'OK':
        status = f"{STATUS_OK} OK"
    elif r['status'] == 'WARN':
        status = f"{STATUS_WARN} {r['error'][:20]}" if r['error'] else f"{STATUS_WARN}"
    else:
        status = f"{STATUS_FAIL} {r['error'][:20]}" if r['error'] else f"{STATUS_FAIL}"
    
    print(f"{outer:<20} | {inner:<20} | {n:>5} | {k:>3} | {circ:>8} | {status}")

# Multi-level results
if multi_results:
    print("\n\nMULTI-LEVEL CONCATENATION:")
    print("-"*100)
    for r in multi_results:
        desc = r['desc'][:50] if r.get('desc') else 'N/A'
        n = str(r['n']) if r['n'] else '-'
        k = str(r['k']) if r['k'] else '-'
        
        if r['status'] == 'OK':
            status = f"{STATUS_OK} OK"
        elif r['status'] == 'WARN':
            status = f"{STATUS_WARN}"
        else:
            status = f"{STATUS_FAIL} {r['error'][:30]}" if r['error'] else f"{STATUS_FAIL}"
        
        print(f"{desc:<50} | n={n:>5} k={k:>3} | {status}")

# Homological product results
if hom_results:
    print("\n\nHOMOLOGICAL PRODUCT:")
    print("-"*100)
    for r in hom_results:
        code_a = r['code_a'][:20] if r.get('code_a') else 'N/A'
        code_b = r['code_b'][:20] if r.get('code_b') else 'N/A'
        n = str(r['n']) if r['n'] else '-'
        k = str(r['k']) if r['k'] else '-'
        
        if r['status'] == 'OK':
            status = f"{STATUS_OK} OK"
        else:
            status = f"{STATUS_FAIL} {r['error'][:30]}" if r['error'] else f"{STATUS_FAIL}"
        
        print(f"{code_a:<20} ⊗ {code_b:<20} | n={n:>5} k={k:>3} | {status}")

# Dual code results
if dual_results:
    print("\n\nDUAL CODES:")
    print("-"*100)
    for r in dual_results:
        orig = r['original'][:30] if r.get('original') else 'N/A'
        n = str(r['n']) if r['n'] else '-'
        k = str(r['k']) if r['k'] else '-'
        
        if r['status'] == 'OK':
            status = f"{STATUS_OK}"
        elif r['status'] == 'WARN':
            status = f"{STATUS_WARN}"
        else:
            status = f"{STATUS_FAIL}"
        
        print(f"Dual({orig:<30}) | n={n:>5} k={k:>3} | {status}")

COMPOSITE CODE SMOKE TEST - COMPREHENSIVE RESULTS

CONCATENATION (ConcatenatedTopologicalCSSCode):
----------------------------------------------------------------------------------------------------
Outer                | Inner                |     n |   k |  Circuit | Status
----------------------------------------------------------------------------------------------------
RotatedSurface_[[9,1 | FourQubit422_[[4,2,2 |    36 |  10 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | C6                   |    54 |  10 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Steane_713           |    63 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Shor_91              |    81 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Hamming_CSS_7        |    63 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Code_832             |     - |   - |        ✗ | ✗ ValueError: Hx Hz^T 
RotatedSurface_[[9,1 | Repetition_3         |    27 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Repetition_5         |    45 |   1 |        ✓ | 